In [23]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Flatten,Conv2D

In [24]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

x_train =  x_train [...,tf.newaxis]
x_test =  x_test [...,tf.newaxis]

In [25]:
type(x_train[0][0,0,0])

numpy.float64

In [26]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(32)
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [27]:
class Model(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.conv1 = Conv2D(32,3,activation="relu")
    self.flatten = Flatten()
    self.dense1 = Dense(128,activation="relu")
    self.dense2 = Dense(10)

  def call(self,x):
     x = self.conv1(x)
     x = self.flatten(x)
     x = self.dense1(x)
     x = self.dense2(x)
     return x

model = Model()

In [28]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [29]:
train_loss = tf.keras.metrics.Mean()
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

test_loss = tf.keras.metrics.Mean()
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [30]:
@tf.function
def train(imagse,labels):
    with tf.GradientTape() as tape:
        pred = model(imagse)
        loss = loss_function(labels,pred)
    gradients = tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels,pred)

In [31]:
@tf.function
def test(imagse,labels):
        pred = model(imagse)
        loss = loss_function(labels,pred)
        test_loss(loss)
        test_accuracy(labels,pred)

In [33]:
epochs = 8
for epoch in range(epochs):
  train_loss.reset_state()
  train_accuracy.reset_state()
  test_loss.reset_state()
  test_accuracy.reset_state()

  for imagse , labels in train_data:
    train(imagse,labels)

  for imagse , labels in test_data:
    test(imagse,labels)

  print("epoch :" ,epoch + 1,
        f"Train Loss :{train_loss.result()}",
        f"Train accuracy:{train_accuracy.result()}",
        f"Test Loss :   {test_loss.result()}",
        f"Test accuracy:{test_accuracy.result()}")


epoch : 1 Train Loss :0.12949919700622559 Train accuracy:0.9609333276748657 Test Loss :   0.07330644130706787 Test accuracy:0.9751999974250793
epoch : 2 Train Loss :0.0420936681330204 Train accuracy:0.9870166778564453 Test Loss :   0.056335125118494034 Test accuracy:0.9825000166893005
epoch : 3 Train Loss :0.020403025671839714 Train accuracy:0.9931666851043701 Test Loss :   0.06978178024291992 Test accuracy:0.9789999723434448
epoch : 4 Train Loss :0.013748398050665855 Train accuracy:0.9952666759490967 Test Loss :   0.05515453219413757 Test accuracy:0.9850999712944031
epoch : 5 Train Loss :0.009379803203046322 Train accuracy:0.9969333410263062 Test Loss :   0.06411313265562057 Test accuracy:0.9835000038146973
epoch : 6 Train Loss :0.006237611174583435 Train accuracy:0.9979000091552734 Test Loss :   0.06672245264053345 Test accuracy:0.9848999977111816
epoch : 7 Train Loss :0.005729336757212877 Train accuracy:0.9979333281517029 Test Loss :   0.07128465920686722 Test accuracy:0.98400002717

In [35]:
import cv2
import numpy as np
image = cv2.imread("image.png")
image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
image = cv2.resize(image,(28,28))
image = image[...,tf.newaxis]
image = image [tf.newaxis,...]
image = image.astype("float32")

prad = model(image)
result = np.argmax(prad)
print(result)

7


In [ ]:
model.export("my_model")
